<a href="https://colab.research.google.com/github/ShyHasVan/IMLO-project/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F

custom_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

training_data = datasets.Flowers102(
    root="data",
    split= "train",
    download=True,
    transform= custom_transform,
    target_transform=Lambda(lambda y: torch.zeros(102, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

test_data = datasets.Flowers102(
    root="data",
    split= "test",
    download=True,
    transform= custom_transform,
    target_transform=Lambda(lambda y: torch.zeros(102, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


for i, (X_Train, y_train) in enumerate(training_data):
  break

conv1 = nn.Conv2d(3, 18, 3, 1, 'same')
conv2 = nn.Conv2d(18, 144, 3, 1, 'same')

x = F.relu(conv1(X_Train))

x = F.max_pool2d(x, 2, 2)
x.shape

x = F.relu(conv2(x))

x = F.max_pool2d(x, 2, 2)
x.shape

Using cpu device


torch.Size([144, 56, 56])

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv2d(3, 18, 3, 1, 'same')
        self.conv2 = nn.Conv2d(18, 144, 3, 1, 'same')
        #Fully Connected Layer
        self.fc1 = nn.Linear(144*56*56, 200)
        self.fc2 = nn.Linear(200, 150)
        self.fc3 = nn.Linear(150, 102)

    def forward(self, X):
      x = self.flatten(X)
      X = F.relu(self.conv1(X))
      X = F.max_pool2d(X,2,2)
      #Second Pass
      X = F.relu(self.conv2(X))
      X = F.max_pool2d(X,2,2)
      #Fully Connected Layers
      X = F.relu(self.fc1(X.view(-1, 144*56*56)))
      X = F.relu(self.fc2(X))
      X = self.fc3(X)
      return F.log_softmax(X, dim=1)


In [ ]:
torch.manual_seed(64)
model  =  NeuralNetwork()
model


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (conv1): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(18, 144, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (fc1): Linear(in_features=451584, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=150, bias=True)
  (fc3): Linear(in_features=150, out_features=102, bias=True)
)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time
start = time.time()

batch_size = 64
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

end = time.time()
total = end - start
print(f"Training took: {total/60} minutes!")

Epoch 1
-------------------------------
loss: 5.549406  [   64/ 1020]


RuntimeError: The size of tensor a (64) must match the size of tensor b (102) at non-singleton dimension 1